# i am beginner in the world of ML and deep leanring. I am eager to interact and learn from the experts.  here below my proposed model to analyze the stroke dataset.
**I reached approx. 97% F1 score on validation set **using SMOTE for oversampling, standardization of 3 features i.e. BMI, AGE and AVG-GLUCOSE-LEVEL and as model I used XGBClassifier.
I thank anyone willing to comment and give feedback on what could be improved in terms of EDA, preprocessing, models, etc.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# this is my first autonomous and indipnendet work after following some Kaggle courses on ML, DL and data visualization.

# very much eager to interact with outher users and read feedbacks/comments to speed up my leanring curve and build up more and more experience.
#  

In [ ]:
datafilepath="../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv"
data=pd.read_csv(datafilepath, index_col='id', parse_dates=True)
import seaborn as sns
X=data.copy()
target_var=X.pop('stroke')

***NaN check***

In [ ]:
for r in data.columns:
    tmp=data[r].isnull().values.any()
    tmp2=data[r].isnull().sum()
    print(r + "=" + str(tmp*1.0) + "// % NaN = " + str(tmp2*1.0 / (1.0*len(data[r]))))

****bmi is only var with NaN. NaN percentage is 4%, low. I replace NaN with mean****

In [ ]:
data['stroke'].eq(0).sum()

****1D data visualization**** 

In [ ]:
data.info()
data.describe()

In [ ]:
from matplotlib import *
numcat = data.select_dtypes(include=['int64', 'float64'])
print(numcat.columns)
for p in (numcat.columns):
    pyplot.figure()
    sns.histplot(data=data, x=p)

In [ ]:
stringscat = data.select_dtypes(include=['object'])
print(stringscat.columns)
for p in (stringscat.columns):
    pyplot.figure()
    sns.histplot(data=data, x=p)

In [ ]:
data_smoke_unkwn = data[data['smoking_status']=='Unknown']
data_smoke_unkwn['stroke'].eq(0).sum()*100/(len(data_smoke_unkwn['stroke']))
data_smoke_unkwn['stroke'].eq(1).sum()*100/(len(data_smoke_unkwn['stroke']))

In [ ]:
selection={'stroke': [1]}
data_withstroke = data[data['stroke']==1]
data_withstroke.info()
data_withOutstroke = data[data['stroke']==0]
data_withOutstroke.info()

In [ ]:
for p in (stringscat.columns):
    pyplot.figure()
    sns.histplot(data=data, x=p, hue='stroke')

In [ ]:
for p in (numcat.columns):
    pyplot.figure()
    sns.histplot(data=data, x=p, hue='stroke')

****search for correlation / ongoing****

In [ ]:
#only numeric
datacorr=data.corr()
sns.heatmap(datacorr, annot=True)
#only numeric after standardization
dataSc= (data - data.mean(axis=0))/data.std(axis=0)
dataSccorr=dataSc.corr()
pyplot.figure()
sns.heatmap(dataSccorr, annot=True)

****data encoding****

In [ ]:
Xcat=X.select_dtypes('object')
Xcat.describe()
from sklearn.preprocessing import LabelEncoder
le_Xcat = X.copy()
le = LabelEncoder()
for p in X.select_dtypes('object'):
    le_Xcat[p] = le.fit_transform(X[p])
le_Xcat.describe()


****imputing NaN****

In [ ]:
from sklearn.impute import SimpleImputer
myimputer = SimpleImputer()
imp_le_Xcat = pd.DataFrame(myimputer.fit_transform(le_Xcat))
print(le_Xcat.info())
featuresName = le_Xcat.columns
print(featuresName)

# handling the imbalanced dataset

In [ ]:
print('% no stroke = ' + str(100*data['stroke'].eq(0).sum() / len(data['stroke'])))
print('% stroke = ' + str(100*data['stroke'].eq(1).sum() / len(data['stroke'])))

****easy approach / undersample the 'no stroke case' so to have 50/50****

In [ ]:
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE

mySampler = SMOTE(random_state=0)
xres, yres = mySampler.fit_resample(imp_le_Xcat,target_var)
xres.columns = featuresName

In [ ]:
xres.info()

In [ ]:
xres.head()

In [ ]:
#standardize bmi avg glucose level and age
stdfeat = ['age', 'avg_glucose_level', 'bmi']
xstd = xres[stdfeat]
xstdn = (xstd - xstd.mean(axis=0)) / (xstd.std(axis=0))
#nonstd_age = xres.drop('age')
#nonstd_avggluc = xres.drop('avg_glucose_level')
#nonstd_bmi = xres.drop('bmi')
#xresn = pd.concat([xres, xstdn], axis=0)
xrescopy=xres.copy()
xrescopy[stdfeat] = xstdn[stdfeat]
xresn = xrescopy

In [ ]:
#from sklearn.preprocessing import StandardScaler
#myScaler = StandardScaler()
#xres_sc = (xres - xres.mean(axis=0)) / xres.std(axis=0)
#xres_sc

****XGB classifier as first investigated approach****


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.utils import shuffle
xtrain, xval, ytrain, yval = train_test_split(xresn, yres, train_size=0.8, test_size=0.2, random_state=0)
from xgboost import XGBClassifier
xtrain = shuffle(xtrain, random_state=1)
ytrain = shuffle(ytrain, random_state=1)
model= XGBClassifier(
    n_estimators=256,
    learning_rate=0.1,
    subsample=0.5,
    use_label_encoder=False,
    random_state=1)
model.fit(xtrain,ytrain)
ypred=model.predict(xval)

In [ ]:
print(classification_report(yval,ypred))
ff=classification_report(yval,ypred)
ff1=f1_score(yval,ypred)
print(ff1)

In [ ]:
xval2, yval2 = shuffle(xval, yval, random_state=21)
ypred2=model.predict(xval2)
print(classification_report(yval2,ypred2))